In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time


# Load the CSV file with extracted features
csv_output_path = r'F:\PhD 2021-2024\Phd-IV Year\VIII\Voice MDVR\DATASET V\SpontaneousDialogue\enhanced_integral_feature_dataset-ii.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Random Forest)
param_dist = {
    'n_estimators': randint(50, 500),  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (Random Forest)
    clf = RandomForestClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (Random Forest)
        clf = RandomForestClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('D2feature_comparison_results_with_80_20_and_k_fold_RF.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.8083
5-Fold Accuracy: 0.7883 ± 0.0480
10-Fold Accuracy: 0.8214 ± 0.0320

Feature Set: Spectrogram
80:20 Accuracy: 0.7461
5-Fold Accuracy: 0.7415 ± 0.0440
10-Fold Accuracy: 0.7623 ± 0.0418

Feature Set: Spectral
80:20 Accuracy: 0.8238
5-Fold Accuracy: 0.7685 ± 0.0205
10-Fold Accuracy: 0.7831 ± 0.0453

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9907 ± 0.0116
10-Fold Accuracy: 1.0000 ± 0.0000

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9917 ± 0.0063
10-Fold Accuracy: 0.9886 ± 0.0177



In [2]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'D2feature_comparison_results_with_80_20_and_k_fold_RF.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data



Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.808290         0.854839      0.654321  0.741259   
1    Spectrogram        0.746114         0.695122      0.703704  0.699387   
2       Spectral        0.823834         0.813333      0.753086  0.782051   
3  Voice_Quality        1.000000         1.000000      1.000000  1.000000   
4   All_Features        1.000000         1.000000      1.000000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.886464   0.606603        0.478318           0.593221   0.191710   
1       0.824846   0.479697        0.504997           0.479670   0.253886   
2       0.879299   0.635915        0.435541           0.634592   0.176166   
3       1.000000   1.000000        0.014597           1.000000   0.000000   
4       1.000000   1.000000        0.118830           1.000000   0.000000   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.055727                 0.028980             0.065556   
1  ...            0.045605                 0.050494             0.085876   
2  ...            0.056594                 0.035180             0.093556   
3  ...            0.000000                 0.000000             0.000000   
4  ...            0.019295                 0.000740             0.033520   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.033986                     0.072685             0.031973   
1                  0.060190                     0.083539             0.041793   
2                  0.043106                     0.093546             0.045275   
3                  0.005649                     0.000000             0.000000   
4                  0.043678                     0.035512             0.017709   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.014353                       3.139247   
1                     0.060775                       1.938984   
2                     0.049867                       2.001601   
3                     0.000000                       0.250270   
4                     0.032314                       5.901531   

   10-Fold Std Dev testing_time  \
0                      0.008682   
1                      0.009254   
2                      0.011645   
3                      0.008559   
4                      0.008566   

                                   Best Params 80:20  
0  {'bootstrap': False, 'max_depth': 30, 'max_fea...  
1  {'bootstrap': True, 'max_depth': 100, 'max_fea...  
2  {'bootstrap': False, 'max_depth': 10, 'max_fea...  
3  {'bootstrap': False, 'max_depth': 10, 'max_fea...  
4  {'bootstrap': False, 'max_depth': 30, 'max_fea...  

[5 rows x 62 columns]

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Load the CSV file with extracted features
csv_output_path = r'F:\PhD 2021-2024\Phd-IV Year\VIII\Voice MDVR\DATASET V\SpontaneousDialogue\enhanced_integral_feature_dataset-ii.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values and infinite values
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (KNN)
param_dist = {
    'n_neighbors': randint(1, 50),  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'p': [1, 2],  # Power parameter for Minkowski distance (1: Manhattan, 2: Euclidean)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Different algorithms to try
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    try:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(y_true, y_pred_proba) if len(np.unique(y_true)) > 1 else 0.5,
            'mcc': matthews_corrcoef(y_true, y_pred),
            'log_loss': log_loss(y_true, y_pred_proba),
            'cohen_kappa': cohen_kappa_score(y_true, y_pred),
            'mae': mean_absolute_error(y_true, y_pred),
            'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1]) if (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1]) != 0 else 0
        }
    except Exception as e:
        print(f"Error calculating metrics: {e}")
        metrics = {k: np.nan for k in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'mcc', 'log_loss', 'cohen_kappa', 'mae', 'specificity']}
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Initialize the classifier with RandomizedSearchCV (KNN)
        clf = KNeighborsClassifier()
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=20,  # Reduced n_iter for testing
            scoring='accuracy', cv=3, verbose=0, random_state=42, n_jobs=1  # Reduced cv and n_jobs=1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_test)
        y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
        metrics['training_time'] = training_time
        metrics['testing_time'] = testing_time
        
        return metrics, random_search.best_params_
    except Exception as e:
        print(f"Error in 80:20 evaluation: {e}")
        return {k: np.nan for k in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'mcc', 'log_loss', 'cohen_kappa', 'mae', 'specificity', 'training_time', 'testing_time']}, {}

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    try:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
        
        fold_metrics = {
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': [],
            'roc_auc': [],
            'mcc': [],
            'log_loss': [],
            'cohen_kappa': [],
            'mae': [],
            'specificity': [],
            'training_time': [],
            'testing_time': []
        }
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Initialize the classifier with RandomizedSearchCV (KNN)
            clf = KNeighborsClassifier()
            random_search = RandomizedSearchCV(
                estimator=clf, param_distributions=param_dist, n_iter=20,
                scoring='accuracy', cv=3, verbose=0, random_state=42, n_jobs=1
            )
            
            # Train the model
            start_train_time = time.time()
            random_search.fit(X_train, y_train)
            end_train_time = time.time()
            training_time = end_train_time - start_train_time
            
            best_clf = random_search.best_estimator_
            
            # Test the model
            start_test_time = time.time()
            y_pred = best_clf.predict(X_val)
            y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
            end_test_time = time.time()
            testing_time = end_test_time - start_test_time
            
            # Calculate performance metrics
            metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
            for key in metrics:
                fold_metrics[key].append(metrics[key])
            fold_metrics['training_time'].append(training_time)
            fold_metrics['testing_time'].append(testing_time)
        
        # Calculate mean and std dev of performance metrics
        mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
        std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
        
        return mean_metrics, std_metrics
    except Exception as e:
        print(f"Error in {k}-fold evaluation: {e}")
        return {k: np.nan for k in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'mcc', 'log_loss', 'cohen_kappa', 'mae', 'specificity', 'training_time', 'testing_time']}, {k: np.nan for k in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'mcc', 'log_loss', 'cohen_kappa', 'mae', 'specificity', 'training_time', 'testing_time']}

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    try:
        print(f"\nProcessing feature group: {group_name}")
        X = data[features]
        y = data['label']
        
        # Check for NaN/infinite values
        if X.isnull().any().any() or np.isinf(X.values).any():
            print(f"Warning: NaN or infinite values found in feature group {group_name}")
            continue
            
        # Evaluate using 80:20 train-test split
        print("Running 80:20 split evaluation...")
        metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
        
        # Evaluate using 5-fold cross-validation
        print("Running 5-fold CV evaluation...")
        mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
        
        # Evaluate using 10-fold cross-validation
        print("Running 10-fold CV evaluation...")
        mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
        
        # Store results for this feature group
        results.append({
            'Feature Set': group_name,
            **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
            **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
            **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
            **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
            **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
            'Best Params 80:20': str(best_params_80_20)
        })
    except Exception as e:
        print(f"Error processing feature group {group_name}: {e}")
        continue

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"\nFeature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")

# Optionally, save the results to a CSV file
results_df.to_csv('D2feature_comparison_results_with_80_20_and_k_fold_KNN.csv', index=False)


Processing feature group: MFCC
Running 80:20 split evaluation...
Running 5-fold CV evaluation...
Running 10-fold CV evaluation...

Processing feature group: Spectrogram
Running 80:20 split evaluation...
Running 5-fold CV evaluation...
Running 10-fold CV evaluation...

Processing feature group: Spectral
Running 80:20 split evaluation...
Running 5-fold CV evaluation...
Running 10-fold CV evaluation...

Processing feature group: Voice_Quality
Running 80:20 split evaluation...
Running 5-fold CV evaluation...
Running 10-fold CV evaluation...

Processing feature group: All_Features
Running 80:20 split evaluation...
Running 5-fold CV evaluation...
Running 10-fold CV evaluation...

Feature Set: MFCC
80:20 Accuracy: 0.8653
5-Fold Accuracy: 0.7591 ± 0.0251
10-Fold Accuracy: 0.7758 ± 0.0264

Feature Set: Spectrogram
80:20 Accuracy: 0.6528
5-Fold Accuracy: 0.6449 ± 0.0137
10-Fold Accuracy: 0.6532 ± 0.0402

Feature Set: Spectral
80:20 Accuracy: 0.6218
5-Fold Accuracy: 0.6740 ± 0.0161
10-Fold Accur

In [9]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'D2feature_comparison_results_with_80_20_and_k_fold_KNN.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.865285         0.848101      0.827160  0.837500   
1    Spectrogram        0.652850         0.634615      0.407407  0.496241   
2       Spectral        0.621762         0.555556      0.493827  0.522876   
3  Voice_Quality        1.000000         1.000000      1.000000  1.000000   
4   All_Features        0.725389         0.705882      0.592593  0.644295   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.927304   0.722651        1.160689           0.722486   0.134715   
1       0.659722   0.264477        0.647742           0.250159   0.347150   
2       0.687280   0.212369        0.982259           0.211363   0.378238   
3       1.000000   1.000000        0.003749           1.000000   0.000000   
4       0.756614   0.427725        3.174190           0.423426   0.274611   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.038300                 0.020091             0.058658   
1  ...            0.060871                 0.061112             0.093493   
2  ...            0.064212                 0.040233             0.097629   
3  ...            0.011563                 0.003699             0.019129   
4  ...            0.068596                 0.040429             0.093406   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.026727                     0.055815             0.026385   
1                  0.039889                     0.086607             0.040190   
2                  0.685020                     0.098443             0.047551   
3                  0.108309                     0.019669             0.009542   
4                  0.815677                     0.095899             0.044015   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.035033                       0.036753   
1                     0.052985                       0.015684   
2                     0.065901                       0.014825   
3                     0.005357                       0.012229   
4                     0.045812                       0.010845   

   10-Fold Std Dev testing_time  \
0                      0.001564   
1                      0.000642   
2                      0.000523   
3                      0.000470   
4                      0.000001   

                                   Best Params 80:20  
0  {'algorithm': 'ball_tree', 'n_neighbors': 4, '...  
1  {'algorithm': 'kd_tree', 'n_neighbors': 22, 'p...  
2  {'algorithm': 'ball_tree', 'n_neighbors': 9, '...  
3  {'algorithm': 'ball_tree', 'n_neighbors': 4, '...  
4  {'algorithm': 'ball_tree', 'n_neighbors': 4, '...  

[5 rows x 62 columns]

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'F:\PhD 2021-2024\Phd-IV Year\VIII\Voice MDVR\DATASET V\SpontaneousDialogue\enhanced_integral_feature_dataset-ii.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Decision Tree)
param_dist = {
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'criterion': ['gini', 'entropy']  # Function to measure the quality of a split
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (Decision Tree)
    clf = DecisionTreeClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (Decision Tree)
        clf = DecisionTreeClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('D2feature_comparison_results_with_80_20_and_k_fold_DecisionTree.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.7098
5-Fold Accuracy: 0.6666 ± 0.0302
10-Fold Accuracy: 0.6740 ± 0.0226

Feature Set: Spectrogram
80:20 Accuracy: 0.7047
5-Fold Accuracy: 0.6885 ± 0.0549
10-Fold Accuracy: 0.7031 ± 0.0517

Feature Set: Spectral
80:20 Accuracy: 0.7150
5-Fold Accuracy: 0.6781 ± 0.0146
10-Fold Accuracy: 0.7341 ± 0.0379

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9833 ± 0.0236
10-Fold Accuracy: 0.9865 ± 0.0219

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9834 ± 0.0129
10-Fold Accuracy: 0.9958 ± 0.0051



In [10]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'D2feature_comparison_results_with_80_20_and_k_fold_DecisionTree.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.709845         0.640449      0.703704  0.670588   
1    Spectrogram        0.704663         0.657895      0.617284  0.636943   
2       Spectral        0.715026         0.666667      0.641975  0.654088   
3  Voice_Quality        1.000000         1.000000      1.000000  1.000000   
4   All_Features        1.000000         1.000000      1.000000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.745756   0.413814    6.025255e+00           0.412353   0.290155   
1       0.743056   0.389020    3.077623e+00           0.388460   0.295337   
2       0.767030   0.412156    5.259377e+00           0.411944   0.284974   
3       1.000000   1.000000    2.220446e-16           1.000000   0.000000   
4       1.000000   1.000000    2.220446e-16           1.000000   0.000000   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.033883                 0.029831             0.047830   
1  ...            0.057106                 0.055868             0.104779   
2  ...            0.049108                 0.034971             0.080126   
3  ...            0.024917                 0.002855             0.043084   
4  ...            0.006048                 0.004268             0.010332   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  2.493218                     0.047911             0.022585   
1                  1.002732                     0.103220             0.051658   
2                  1.970377                     0.079151             0.037883   
3                  0.135812                     0.044362             0.021875   
4                  0.147945                     0.010441             0.005090   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.029711                       0.060021   
1                     0.070330                       0.031466   
2                     0.042581                       0.017401   
3                     0.032758                       0.039577   
4                     0.007143                       0.048682   

   10-Fold Std Dev testing_time  \
0                      0.000311   
1                      0.000837   
2                      0.000726   
3                      0.000985   
4                      0.000466   

                                   Best Params 80:20  
0  {'criterion': 'gini', 'max_depth': 50, 'max_fe...  
1  {'criterion': 'entropy', 'max_depth': 10, 'max...  
2  {'criterion': 'gini', 'max_depth': 30, 'max_fe...  
3  {'criterion': 'entropy', 'max_depth': 20, 'max...  
4  {'criterion': 'entropy', 'max_depth': 100, 'ma...  

[5 rows x 62 columns]

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'F:\PhD 2021-2024\Phd-IV Year\VIII\Voice MDVR\DATASET V\SpontaneousDialogue\enhanced_integral_feature_dataset-ii.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (AdaBoost)
param_dist = {
    'n_estimators': randint(50, 500),  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Learning rate
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (AdaBoost)
    clf = AdaBoostClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (AdaBoost)
        clf = AdaBoostClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('D2feature_comparison_results_with_80_20_and_k_fold_AdaBoost.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.7409
5-Fold Accuracy: 0.7445 ± 0.0256
10-Fold Accuracy: 0.7788 ± 0.0449

Feature Set: Spectrogram
80:20 Accuracy: 0.7358
5-Fold Accuracy: 0.7218 ± 0.0337
10-Fold Accuracy: 0.7248 ± 0.0529

Feature Set: Spectral
80:20 Accuracy: 0.7513
5-Fold Accuracy: 0.7435 ± 0.0192
10-Fold Accuracy: 0.7643 ± 0.0380

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9990 ± 0.0021
10-Fold Accuracy: 1.0000 ± 0.0000

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 1.0000 ± 0.0000
10-Fold Accuracy: 0.9885 ± 0.0344



In [12]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'D2feature_comparison_results_with_80_20_and_k_fold_AdaBoost.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.740933         0.696203      0.679012  0.687500   
1    Spectrogram        0.735751         0.708333      0.629630  0.666667   
2       Spectral        0.751295         0.732394      0.641975  0.684211   
3  Voice_Quality        1.000000         1.000000      1.000000  1.000000   
4   All_Features        1.000000         1.000000      1.000000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.818563   0.466426        0.676231           0.466320   0.259067   
1       0.790234   0.451172        0.659467           0.449034   0.264249   
2       0.810185   0.483382        0.679913           0.480543   0.248705   
3       1.000000   1.000000        0.555309           1.000000   0.000000   
4       1.000000   1.000000        0.537773           1.000000   0.000000   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.077799             3.873140e-02             0.095446   
1  ...            0.078648             6.164566e-02             0.114132   
2  ...            0.054880             4.279219e-02             0.081407   
3  ...            0.000000             3.510833e-17             0.000000   
4  ...            0.044000             4.017857e-03             0.070631   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.008301                     0.101610             0.044947   
1                  0.016581                     0.114277             0.052894   
2                  0.010425                     0.081630             0.038001   
3                  0.030915                     0.000000             0.000000   
4                  0.025314                     0.072000             0.034375   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.041090                      13.675924   
1                     0.049443                       1.759437   
2                     0.037581                       4.276775   
3                     0.000000                       1.504005   
4                     0.016071                     427.911205   

   10-Fold Std Dev testing_time  \
0                      0.032590   
1                      0.162637   
2                      0.218599   
3                      0.239263   
4                      0.258270   

                                   Best Params 80:20  
0  {'algorithm': 'SAMME.R', 'learning_rate': 0.5,...  
1  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  
2  {'algorithm': 'SAMME.R', 'learning_rate': 0.5,...  
3  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  
4  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  

[5 rows x 62 columns]

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'F:\PhD 2021-2024\Phd-IV Year\VIII\Voice MDVR\DATASET V\SpontaneousDialogue\enhanced_integral_feature_dataset-ii.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Random Forest)
param_dist_rf = {
    'n_estimators': randint(50, 500),  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Parameter grid for RandomizedSearchCV (Decision Tree)
param_dist_dt = {
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'criterion': ['gini', 'entropy']  # Function to measure the quality of a split
}

# Parameter grid for RandomizedSearchCV (AdaBoost)
param_dist_ab = {
    'n_estimators': randint(50, 500),  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Learning rate
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Define base models
    rf = RandomForestClassifier(random_state=42)
    dt = DecisionTreeClassifier(random_state=42)
    ab = AdaBoostClassifier(random_state=42)
    
    # Define meta-classifier
    meta_classifier = LogisticRegression(random_state=42)
    
    # Define stacking classifier
    stacking_clf = StackingClassifier(
        estimators=[('rf', rf), ('dt', dt), ('ab', ab)],
        final_estimator=meta_classifier,
        stack_method='auto',
        n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    stacking_clf.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    # Test the model
    start_test_time = time.time()
    y_pred = stacking_clf.predict(X_test)
    y_pred_proba = stacking_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Define base models
        rf = RandomForestClassifier(random_state=42)
        dt = DecisionTreeClassifier(random_state=42)
        ab = AdaBoostClassifier(random_state=42)
        
        # Define meta-classifier
        meta_classifier = LogisticRegression(random_state=42)
        
        # Define stacking classifier
        stacking_clf = StackingClassifier(
            estimators=[('rf', rf), ('dt', dt), ('ab', ab)],
            final_estimator=meta_classifier,
            stack_method='auto',
            n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        stacking_clf.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        # Test the model
        start_test_time = time.time()
        y_pred = stacking_clf.predict(X_val)
        y_pred_proba = stacking_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()}
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('D2feature_comparison_results_with_80_20_and_k_fold_Stacking.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.8238
5-Fold Accuracy: 0.7446 ± 0.0455
10-Fold Accuracy: 0.7497 ± 0.0205

Feature Set: Spectrogram
80:20 Accuracy: 0.7358
5-Fold Accuracy: 0.7529 ± 0.0425
10-Fold Accuracy: 0.7643 ± 0.0410

Feature Set: Spectral
80:20 Accuracy: 0.8135
5-Fold Accuracy: 0.7923 ± 0.0185
10-Fold Accuracy: 0.7778 ± 0.0388

Feature Set: Voice_Quality
80:20 Accuracy: 0.9896
5-Fold Accuracy: 1.0000 ± 0.0000
10-Fold Accuracy: 1.0000 ± 0.0000

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9927 ± 0.0053
10-Fold Accuracy: 0.9990 ± 0.0031



In [14]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'D2feature_comparison_results_with_80_20_and_k_fold_Stacking.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.823834         0.840580      0.716049  0.773333   
1    Spectrogram        0.735751         0.692308      0.666667  0.679245   
2       Spectral        0.813472         0.800000      0.740741  0.769231   
3  Voice_Quality        0.989637         1.000000      0.975309  0.987500   
4   All_Features        1.000000         1.000000      1.000000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.893519   0.636193        0.415629           0.630767   0.176166   
1       0.825287   0.454946        0.506768           0.454712   0.264249   
2       0.876102   0.614376        0.435081           0.613097   0.186528   
3       1.000000   0.978876        0.021874           0.978653   0.010363   
4       1.000000   1.000000        0.018997           1.000000   0.000000   

   ...  10-Fold Std Dev recall  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  \
0  ...                0.060057            0.055426                 0.018328   
1  ...                0.058850            0.053752                 0.057282   
2  ...                0.071010            0.057741                 0.035481   
3  ...                0.000000            0.000000                 0.000000   
4  ...                0.000000            0.003704                 0.000402   

   10-Fold Std Dev mcc  10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  \
0             0.042458                  0.016267                     0.052758   
1             0.087442                  0.041140                     0.085989   
2             0.084030                  0.020667                     0.084091   
3             0.000000                  0.004975                     0.000000   
4             0.006339                  0.016824                     0.006406   

   10-Fold Std Dev mae  10-Fold Std Dev specificity  \
0             0.020472                     0.018819   
1             0.040985                     0.046773   
2             0.038808                     0.038468   
3             0.000000                     0.000000   
4             0.003125                     0.005357   

   10-Fold Std Dev training_time  10-Fold Std Dev testing_time  
0                       0.963536                      0.115494  
1                       0.304748                      0.066917  
2                       0.359951                      0.124607  
3                       0.335428                      0.091039  
4                       0.855552                      0.144350  

[5 rows x 61 columns]